In [1]:
import evotorch.operators
import torch
from torch import nn
from evotorch.decorators import pass_info
from evotorch.neuroevolution import GymNE
from evotorch.logging import StdOutLogger
from evotorch.algorithms import Cosyne

# The decorator `@pass_info` tells the problem class `GymNE`
# to pass information regarding the gym environment via keyword arguments
# such as `obs_length` and `act_length`.
@pass_info
class LinearPolicy(nn.Module):
    def __init__(
            self,
            obs_length: int,  # Number of observations from the environment
            act_length: int,  # Number of actions of the environment
            bias: bool = True,  # Whether the policy should use biases
            **kwargs  # Anything else that is passed
    ):
        super().__init__()  # Always call super init for nn Modules
        self.linear = nn.Linear(obs_length, act_length, bias=bias)

    def forward(self, obs: torch.Tensor) -> torch.Tensor:
        # Forward pass of model simply applies linear layer to observations
        return self.linear(obs)

# Setting up the GymNE problem
problem = GymNE(
    env="LunarLanderContinuous-v3",  # Name of the environment
    network=LinearPolicy,  # Linear policy that we defined earlier
    network_args={'bias': False},  # Linear policy should not use biases
    num_actors=4,  # Use 4 available CPUs. You can modify this value, or use 'max' to exploit all available CPUs
    observation_normalization=False,  # Observation normalization was not used in Lunar Lander experiments
)

# Setting up the PGPE searcher
radius_init = 4.5  # (approximate) radius of initial hypersphere that we will sample from
max_speed = radius_init / 15.  # Rule-of-thumb from the paper
center_learning_rate = max_speed / 2.

searcher = Cosyne(
    problem,
    num_elites=1,
    popsize=50,
    tournament_size=4,
    mutation_stdev=0.3,
    mutation_probability=0.5,
    permute_all=True,
)


if __name__ == '__main__':
    StdOutLogger(searcher)
    searcher.run(50)

[2024-11-23 16:49:54] INFO     < 8696> evotorch.core: Instance of `GymNE` (id:1997685684464) -- The `dtype` for the problem's decision variables is set as torch.float32
[2024-11-23 16:49:54] INFO     < 8696> evotorch.core: Instance of `GymNE` (id:1997685684464) -- `eval_dtype` (the dtype of the fitnesses and evaluation data) is set as torch.float32
[2024-11-23 16:49:54] INFO     < 8696> evotorch.core: Instance of `GymNE` (id:1997685684464) -- The `device` of the problem is set as cpu
[2024-11-23 16:49:54] INFO     < 8696> evotorch.core: Instance of `GymNE` (id:1997685684464) -- The number of actors that will be allocated for parallelized evaluation is 4
[2024-11-23 16:49:54] INFO     < 8696> evotorch.core: Instance of `GymNE` (id:1997685684464) -- Number of GPUs that will be allocated per actor is None


2024-11-23 16:50:02,028	ERROR services.py:1350 -- Failed to start the dashboard , return code 1
2024-11-23 16:50:02,032	ERROR services.py:1375 -- Error should be written to 'dashboard.log' or 'dashboard.err'. We are printing the last 20 lines for you. See 'https://docs.ray.io/en/master/ray-observability/user-guides/configure-logging.html#logging-directory-structure' to find where the log file is.
2024-11-23 16:50:02,065	ERROR services.py:1419 -- 
The last 20 lines of C:\Users\THELAP~1\AppData\Local\Temp\ray\session_2024-11-23_16-49-56_861153_8696\logs\dashboard.log (it contains the error message from the dashboard): 
  File "<frozen importlib._bootstrap>", line 1360, in _find_and_load
  File "<frozen importlib._bootstrap>", line 1331, in _find_and_load_unlocked
  File "<frozen importlib._bootstrap>", line 935, in _load_unlocked
  File "<frozen importlib._bootstrap_external>", line 995, in exec_module
  File "<frozen importlib._bootstrap>", line 488, in _call_with_frames_removed
  File 

(raylet) The node with node id: 430abdcd7403ed249fbab55aa86f50076f12da229fa4d2c3924ab4af and address: 127.0.0.1 and node name: 127.0.0.1 has been marked dead because the detector has missed too many heartbeats from it. This can happen when a 	(1) raylet crashes unexpectedly (OOM, etc.) 
	(2) raylet has lagging heartbeats due to slow network or busy workload.


(raylet) [2024-11-23 16:50:11,769 E 27060 20236] (raylet.exe) agent_manager.cc:83: The raylet exited immediately because one Ray agent failed, agent_name = dashboard_agent/15724.
(raylet) The raylet fate shares with the agent. This can happen because
(raylet) - The version of `grpcio` doesn't follow Ray's requirement. Agent can segfault with the incorrect `grpcio` version. Check the grpcio version `pip freeze | grep grpcio`.
(raylet) - The agent failed to start because of unexpected error or port conflict. Read the log `cat /tmp/ray/session_latest/logs/{dashboard_agent|runtime_env_agent}.log`. You can find the log file structure here https://docs.ray.io/en/master/ray-observability/user-guides/configure-logging.html#logging-directory-structure.
(raylet) - The agent is killed by the OS (e.g., out of memory).
(raylet) [2024-11-23 16:50:12,036 E 27060 27036] (raylet.exe) raylet.cc:175: Raylet failed to accept new connection: The I/O operation has been aborted because of either a thread exi

ActorDiedError: The actor died unexpectedly before finishing this task.
	class_name: EvaluationActor
	actor_id: 37be9e609e28265ad0d9328901000000
	pid: 19808
	namespace: c5e2ef7c-3031-4e90-a9c3-c09df80d45b3
	ip: 127.0.0.1
The actor is dead because its owner has died. Owner Id: 01000000ffffffffffffffffffffffffffffffffffffffffffffffff Owner Ip address: 127.0.0.1 Owner worker exit type: SYSTEM_ERROR Worker exit detail: Owner's node has crashed.
The actor never ran - it was cancelled before it started running.